<a href="https://colab.research.google.com/github/GremlinAssassin/AssemblyInfer/blob/main/DirectInjection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This cell sets up the script with the python packages that we both need (numpy and pandas) and thoses that are useful for the way we have designed this script (pathlib) and ones that can make the data look nice (matplotlib and seaborn). We are only going to look at the MSexperiment class from assembly Infer and the ripper converts mzML files into a Json format.

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from assemblyinfer import MSExperiment

import mzmlripper.extractor as ripper


Next is the the code that will tell this script where the data you you would like to look at is DATA_FOLDER and where the files should go when we are done. It has been setup so that the finiahed filesand plots will go into the same folder

In [ ]:
DATA_FOLDER = 'D:/Please/change/this'

DATA_OUTPUT = DATA_FOLDER

This is the bit that the 'ripper' will do. We are converting mZML files into JSon files. It allows us easier access to the data inside the files. By doing it this way we also check to see if the corresponding JSON file already exsists. If it does then it will not reconvert the file saving a lot of time.

This will check all folders and subfolders in a specific location. If you do NOT want to look in subfolders change rglob to glob.


In [ ]:
for mzml in list(Path(DATA_FOLDER).rglob("*.mzml")):
    
    if not Path(DATA_FOLDER, f'ripper_{mzml.stem}.json').exists():
        
        ripper.process_mzml_file(
            filename=str(mzml),
            out_dir=DATA_FOLDER)